In [3]:
import requests
import os
import gradio as gr
from fastapi import FastAPI, File, UploadFile
import uvicorn
import shutil

In [4]:
# Set up API Key and Voice ID (Replace with your actual values)
ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY", "sk_b4c0a174eb8a69f6df38f4fccba0f7d5cbffd76956aaac84")
VOICE_ID = "21m00Tcm4TlvDq8ikWAM"  # Get this from ElevenLabs after uploading a sample

def clone_voice(text):
    url = f"https://api.elevenlabs.io/v1/text-to-speech/{VOICE_ID}"
    headers = {
        "Authorization": f"Bearer {ELEVENLABS_API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "text": text,
        "voice_settings": {"stability": 0.5, "similarity_boost": 0.75}
    }

    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        audio_path = "generated_voice.mp3"
        with open(audio_path, "wb") as f:
            f.write(response.content)
        return audio_path  # Return path to generated audio
    else:
        return f"Error: {response.text}"

In [5]:
# Gradio UI: Upload Audio & Enter Text
def process_audio(audio_file, text):
    return clone_voice(text)  # Uses stored voice ID instead of processing audio file

iface = gr.Interface(
    fn=process_audio,
    inputs=[gr.Audio(type="filepath"), "text"],
    outputs="audio",
    title="Voice Cloning App"
)


In [6]:
# FastAPI Server
app = FastAPI()

@app.post("/clone-voice/")
async def clone_voice_api(file: UploadFile = File(...), text: str = "Hello, this is your cloned voice."):
    file_location = f"temp_{file.filename}"
    with open(file_location, "wb") as buffer:
        shutil.copyfileobj(file.file, buffer)

    generated_audio = clone_voice(text)  # Uses stored ElevenLabs voice ID

    return {"audio_file": generated_audio}

# Run FastAPI
def run_fastapi():
    uvicorn.run(app, host="0.0.0.0", port=8000)


In [7]:
# Launch Gradio & FastAPI
if __name__ == "__main__":
    import threading

    threading.Thread(target=run_fastapi).start()
    iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://395599c7f1537a2641.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
